In [2]:
topsis_code = r'''
import sys, os, csv, math

def check_argument_count():
    if len(sys.argv) != 5:
        print("Error: Incorrect number of parameters.")
        print("Usage : python topsis.py <InputFile> <Weights> <Impacts> <OutputFile>")
        sys.exit(1)

def check_file_exists(path):
    if not os.path.isfile(path):
        print(f"Error: Input file '{path}' not found.")
        sys.exit(1)

def read_input(path):
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        rows = [row for row in reader if any(cell.strip() for cell in row)]
    if len(rows) < 2:
        print("Error: Input file must have a header and at least one data row.")
        sys.exit(1)
    return rows[0], rows[1:]

def check_min_columns(header):
    if len(header) < 3:
        print("Error: Input file must contain three or more columns.")
        sys.exit(1)

def check_numeric(data_rows):
    for r_idx, row in enumerate(data_rows, start=2):
        for c_idx in range(1, len(row)):
            try:
                float(row[c_idx])
            except ValueError:
                print(f"Error: Non-numeric value '{row[c_idx].strip()}' in row {r_idx}, column {c_idx+1}.")
                sys.exit(1)

def parse_weights(weight_str, n_criteria):
    parts = [w.strip() for w in weight_str.split(",")]
    if len(parts) != n_criteria:
        print(f"Error: Number of weights ({len(parts)}) does not match criteria columns ({n_criteria}).")
        sys.exit(1)
    weights = []
    for p in parts:
        try:
            weights.append(float(p))
        except ValueError:
            print(f"Error: '{p}' is not a valid weight.")
            sys.exit(1)
    return weights

def parse_impacts(impact_str, n_criteria):
    parts = [i.strip() for i in impact_str.split(",")]
    if len(parts) != n_criteria:
        print(f"Error: Number of impacts ({len(parts)}) does not match criteria columns ({n_criteria}).")
        sys.exit(1)
    for p in parts:
        if p not in ("+", "-"):
            print(f"Error: Invalid impact '{p}'. Must be '+' or '-'.")
            sys.exit(1)
    return parts

def topsis(matrix, weights, impacts):
    rows = len(matrix)
    cols = len(matrix[0])
    norm = [[0.0]*cols for _ in range(rows)]
    for j in range(cols):
        denom = math.sqrt(sum(matrix[i][j]**2 for i in range(rows)))
        for i in range(rows):
            norm[i][j] = matrix[i][j]/denom if denom != 0 else 0.0
    v = [[norm[i][j]*weights[j] for j in range(cols)] for i in range(rows)]
    a_pos, a_neg = [0.0]*cols, [0.0]*cols
    for j in range(cols):
        col = [v[i][j] for i in range(rows)]
        if impacts[j] == "+":
            a_pos[j], a_neg[j] = max(col), min(col)
        else:
            a_pos[j], a_neg[j] = min(col), max(col)
    s_pos = [math.sqrt(sum((v[i][j]-a_pos[j])**2 for j in range(cols))) for i in range(rows)]
    s_neg = [math.sqrt(sum((v[i][j]-a_neg[j])**2 for j in range(cols))) for i in range(rows)]
    scores = []
    for i in range(rows):
        total = s_pos[i]+s_neg[i]
        scores.append(round((s_neg[i]/total)*100, 2) if total != 0 else 0.0)
    sorted_idx = sorted(range(rows), key=lambda i: scores[i], reverse=True)
    ranks = [0]*rows
    for rank_val, idx in enumerate(sorted_idx, start=1):
        ranks[idx] = rank_val
    return scores, ranks

def write_output(header, names, matrix, scores, ranks, out_path):
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(list(header)+["Topsis Score","Rank"])
        for i in range(len(names)):
            writer.writerow([names[i]]+matrix[i]+[scores[i], ranks[i]])

def main():
    check_argument_count()
    input_file, weight_str, impact_str, output_file = sys.argv[1], sys.argv[2], sys.argv[3], sys.argv[4]
    check_file_exists(input_file)
    header, data_rows = read_input(input_file)
    check_min_columns(header)
    check_numeric(data_rows)
    n_criteria = len(header)-1
    weights = parse_weights(weight_str, n_criteria)
    impacts = parse_impacts(impact_str, n_criteria)
    names  = [row[0].strip() for row in data_rows]
    matrix = [[float(row[c]) for c in range(1, len(row))] for row in data_rows]
    scores, ranks = topsis(matrix, weights, impacts)
    write_output(header, names, matrix, scores, ranks, output_file)
    print(f"TOPSIS completed. Results saved to '{output_file}'.")

if __name__ == "__main__":
    main()
'''

with open("topsis.py", "w") as f:
    f.write(topsis_code)

print("✅ topsis.py saved.")

✅ topsis.py saved.


In [3]:
import csv

header = ["Fund Name", "P1", "P2", "P3", "P4", "P5"]
rows = [
    ["M1", 0.67, 0.45, 6.5, 42.6, 12.56],
    ["M2", 0.60, 0.36, 3.6, 53.3, 14.47],
    ["M3", 0.82, 0.67, 3.8, 63.1, 17.10],
    ["M4", 0.60, 0.36, 3.5, 69.2, 18.42],
    ["M5", 0.76, 0.58, 4.8, 43.0, 12.29],
    ["M6", 0.69, 0.48, 6.6, 48.7, 14.12],
    ["M7", 0.79, 0.62, 4.8, 59.2, 16.35],
    ["M8", 0.84, 0.71, 6.5, 34.5, 10.64],
]

with open("data.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)

print("✅ data.csv saved.")

✅ data.csv saved.


In [4]:
import subprocess

result = subprocess.run(
    ["python", "topsis.py", "data.csv", "1,1,1,1,1", "+,+,-,+,+", "output-result.csv"],
    capture_output=True, text=True
)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

TOPSIS completed. Results saved to 'output-result.csv'.



In [5]:
import pandas as pd

df = pd.read_csv("output-result.csv")
print(df.to_string(index=False))


Fund Name   P1   P2  P3   P4    P5  Topsis Score  Rank
       M1 0.67 0.45 6.5 42.6 12.56         21.52     8
       M2 0.60 0.36 3.6 53.3 14.47         46.80     4
       M3 0.82 0.67 3.8 63.1 17.10         86.15     1
       M4 0.60 0.36 3.5 69.2 18.42         58.85     3
       M5 0.76 0.58 4.8 43.0 12.29         45.08     5
       M6 0.69 0.48 6.6 48.7 14.12         32.66     7
       M7 0.79 0.62 4.8 59.2 16.35         69.55     2
       M8 0.84 0.71 6.5 34.5 10.64         41.42     6


In [6]:
from google.colab import files

files.download("output-result.csv")
print("⬇️  output-result.csv downloaded.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️  output-result.csv downloaded.
